In [10]:
#test in real world
import sys
sys.path.append('..')

from src.includes import *


home_path = '/home/swj/VD/'
code_type = 'C++'
data_set_name = 'VLC'
#CVE-2021-25804 : OK
#CVE-2021-38114 : OK
CVE_code = 'CVE-2021-25803'
data_path = home_path + 'data/' + 'TEST/' + code_type + '/' + data_set_name + '/'
print(data_path)
model_name = 'blstm'
ways = 'myway'

model_path = '/home/swj/VD/trained_model/' + data_set_name + '/' + model_name + '_' + ways + '_model.h5'
tokenizer_saved_path = '/home/swj/VD/embedding/'
#change this


if ways == 'AST':
    all_data_path = home_path + 'data/ALL/'
    all_data_list, all_data_list_id = loadAST(all_data_path)
    total_list, total_list_id = loadAST(data_path)
else:
    all_data_path = home_path + 'data/Six_projects/'
    all_data_list, all_data_list_id = loadData(all_data_path)
    total_list, total_list_id = loadData(data_path)
print(len(total_list))
verbose("Perform tokenization ....")
Tokenization(all_data_list)
verbose("Tokenization completed!")
verbose("-------------------------------------------------------")   
verbose("Perform code embedding ....")
embedding_model = Embedding_Model(config)
'''total_sequnces: variable lengths of number of samples vectors'''
'''word_index: a map of word to vectors'''
total_sequences, word_index = embedding_model.LoadTokenizer(total_list)
embedding_model.TrainWordToVec(total_list)
embedding_matrix, embedding_dim = embedding_model.ApplyWordToVec(word_index)
weights=torch.from_numpy(embedding_matrix)

#obtain representations
def ObtainRepresentations_by_batch_size(input_sequences, feature_model, BATCH_SIZE):
    num_batches_per_epoch = int((len(input_sequences) - 1) / BATCH_SIZE) + 1
    data_size = len(input_sequences)
    representations_total = []
    model = feature_model
    for batch_num in range(num_batches_per_epoch):
        start_index = batch_num * BATCH_SIZE
        end_index = min((batch_num + 1) * BATCH_SIZE, data_size)
        print ("-------start_index------------")
        print (start_index)
        print ("-------end_index------------")
        print (end_index)
        print(input_sequences[start_index: end_index].shape)
        representations = model(input_sequences[start_index: end_index])
        representations_total = representations_total + representations.tolist()
    return np.asarray(representations_total)


if model_name == 'blstm':
    net = BLSTMnet(weights).to(device)
elif model_name == 'dlstm':
    net = BPLSTMnet(weights).to(device)

    
#change this

net.load_state_dict(torch.load(model_path))
net.eval()

feature_model = None
if model_name == 'blstm':
    feature_model = BLSTM_Feature_Net(net, weights).to(device)
elif model_name == 'dlstm':
    feature_model = BPLSTM_Feature_Net(net, weights).to(device)

feature_model.eval()
tokenizer = LoadToknizer(tokenizer_saved_path + 'tokenizer.pickle')


# if vulnerability == 'vul':
#     data_list, data_list_id = loadData(data_path + 'Vulnerable_functions/')
# else: 
#     data_list, data_list_id = loadData(data_path + 'Non_vulnerable_functions/')

data_list, data_list_id = getCFilesFromText(data_path)
data_sequence = tokenizer.texts_to_sequences(data_list)
data_list_pad = padding(data_sequence)
data_list_pad = torch.Tensor(data_list_pad).long().to(device)
start_time = time.time()
obtained_repre = ObtainRepresentations_by_batch_size(data_list_pad, feature_model, 128)
end_time = time.time()
print("Saving the obtained representations....") 
print('uisng' + str(end_time - start_time) + 's')
# if not os.path.exists(repre_saved_path): os.makedirs(repre_saved_path)

# if vulnerability == 'vul':
#     SavedPickle(repre_saved_path + "Vul.pkl", obtained_repre)
# else:
#     SavedPickle(repre_saved_path + "NonVul.pkl", obtained_repre)

# print("The obtained representations are saved in: " + str(repre_saved_path) + ".")



/home/swj/VD/data/TEST/C++/VLC/
[INFO] Loading data from /home/swj/VD/src//home/swj/VD/data/Six_projects/....
[INFO] The length of the loaded data list is : 39944
[INFO] Loading data from /home/swj/VD/src//home/swj/VD/data/TEST/C++/VLC/....
[INFO] The length of the loaded data list is : 10
10
[INFO] Perform tokenization ....
[INFO] Tokenization completed!
[INFO] -------------------------------------------------------
[INFO] Perform code embedding ....
----------------------------------------
Start training the Word2Vec model. Please wait.. 
Model training completed!
----------------------------------------
The trained word2vec model: 
Word2Vec(vocab=242, vector_size=100, alpha=0.025)
-------------------------------------------------------
Loading trained Word2vec model. 
The trained word2vec model: 
<_io.TextIOWrapper name='/home/swj/VD/embedding/w2v_model.txt' mode='r' encoding='UTF-8'>
Found 243 word vectors.
-------start_index------------
0
-------end_index------------
10
torch.Size

In [11]:
model_path = home_path + 'trained_model' + '/' + model_name + '_' + data_set_name + '_' + ways + '_RF.joblib'
clf = joblib.load(model_path)
y_predict = clf.predict(obtained_repre)
for i in range(len(y_predict)):
    print(data_list_id[i] + ': ' + str(y_predict[i]))
print('# of vul detected: ' + str(sum(y_predict)))

CVE-2021-25803.c: 0
CVE-2021-38114-fix.c: 0
CVE-2021-25801.c: 0
CVE-2021-25803-fix.c: 0
CVE-2021-25804-fix.c: 0
CVE-2021-25801-fix.c: 0
CVE-2021-38114.c: 0
CVE-2021-25802-fix.c: 0
CVE-2021-25802.c: 0
CVE-2021-25804.c: 0
# of vul detected: 0


In [ ]:
model_path = home_path + 'trained_model' + '/' + 'BLSTM_RF.joblib'
clf = joblib.load(model_path)
y_predict = clf.predict(obtained_repre)
for i in range(len(y_predict)):
    print(data_list_id[i] + ': ' + str(y_predict[i]))
print('# of vul detected: ' + str(sum(y_predict)))